# 정치 뉴스를 연관분석 돌리면 너무 시간이 걸려서 최빈 단어만 뽑아 연관분석함

In [1]:
import requests
from bs4 import BeautifulSoup
rss_url = 'https://rss.joins.com/joins_politics_list.xml'
# rss_url = 'https://rss.joins.com/joins_star_list.xml'
money_response = requests.get(rss_url)
money_soup = BeautifulSoup(money_response.content, "xml")
link_list = money_soup.select("item > link")
link_list = [l.text for l in link_list]
# link_list

In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()
res = requests.get('https://news.joins.com/article/24029441?cloc=rss-news-politics')
sou = BeautifulSoup(res.content, "html.parser")
article = sou.select_one('div#article_body').text
tagged_text = kkma.pos(article, 56)
nouns_list = [t[0] for t in tagged_text if ((t[1]=='NNG') | (t[1]=='NNP')) & (len(t[0])>1) & (t[0]!='기자')]
print(nouns_list[:10])

In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()
news = []
for link in link_list:
    news_response = requests.get(link)
    news_soup = BeautifulSoup(news_response.content, "html.parser")
    element = news_soup.select_one('div#article_body')
    # 두가지 type이 있어서 나눔
    if element :
        article = element.text
    else:
        article = news_soup.select_one('p#leadCaption').text
        print(link, False)
    tagged_text = kkma.pos(article, 56)
    nouns_list = [t[0] for t in tagged_text if (t[1]=='NNG') & (len(t[0])>1) & (t[0]!='기자')] # 일반 명사만 추출
#     nouns_list = [t[0] for t in tagged_text if (t[1]=='NNP') & (len(t[0])>1) & (t[0]!='기자')] 고유명사만
    if len(nouns_list)>0 :
        news.append(nouns_list)
len(news)

In [ ]:
temp = []
for news_item in news:
    temp.append(' '.join(news_item))
    word_list = ' '.join(temp)
len(word_list), len(set(word_list))

In [ ]:
import pandas as pd

wordcount = pd.Series(dtype=np.int64)
words = word_list.split()
for word in words:
    if word in wordcount:
        wordcount[word] += 1
    else:
        wordcount[word] = 1
wordcount.sort_values(ascending=False, inplace=True)
freq_word = list(wordcount.head(10).index)
print(freq_word)

In [ ]:
temp = []
for news_item in news:
    t = [word for word in freq_word if word in news_item]
    if len(t)> 0:
        temp.append(t)
temp

In [ ]:
from apyori import apriori
rules = apriori(temp, minsupport=0.3, min_confience=0.2)
result = list(rules)

In [ ]:
for r in result:
    print(r, end="\m\m")

In [ ]:
! jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [ ]:
for r in result:
    print(r, end="\m\m")

In [ ]:
result[0]

In [ ]:
result[100]

In [ ]:
import pandas as pd
result_df =  pd.DataFrame(None, 
                    columns=['lhs','rhs','support','confidence','lift'])
index = 0
for row in result:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ','.join(x for x in item[0])
        rhs = ','.join(x.strip() for x in item[1])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            result_df.loc[index]=[lhs, rhs, support, confidence, lift]
            index += 1
result_df